In [9]:
import pandas as pd
import numpy as np
import requests

from bs4 import BeautifulSoup

In [12]:
# Official link provided in Assignment page
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# Due to changes in webpage, we will be using old revision of the above link
url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969"


In [13]:
canada_postal_codes = "PostalCode;Borough;Neighborhood\n" # Columns for our csv file
filename = "canada_postal_codes.csv"
r = requests.get(url)
soup = BeautifulSoup(r.text, "html.parser")
table = soup.find('table', {'class': 'wikitable sortable'}) # Find table with class name as 'wikitable, sortable'
for row in table.find_all('tr')[1:]:  # iterate over each table row
  code, borough, neighbor = [col.text.strip() for col in row.find_all('td')]  # extract table values
  if borough != "Not assigned":
    if neighbor == "Not assigned":
      neighbor = borough
    canada_postal_codes += f'{code};{borough};{neighbor}\n' # insert values into the table
with open(filename, "w") as f:  # create a csv file and insert records to the file
  f.write(canada_postal_codes)

In [14]:
df = pd.read_csv(filename, ';')
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [15]:
len(df.PostalCode.unique())

103

In [16]:
df.shape

(103, 3)

In [17]:
geospatial_df = pd.read_csv("https://cocl.us/Geospatial_data")
geospatial_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
!pip install geocoder -q
import geocoder

In [19]:
data = []
i = 1
for code in df.PostalCode:
  print('\rProcessing:', i, end="")
  query = '{}, Toronto, Ontario'.format(code)
  row = [code]
  coders = [geocoder.arcgis, geocoder.geocodefarm, geocoder.komoot, geocoder.location, geocoder.osm]
  for c in coders:
    try:
      values = c(query).latlng
    except:
      values = [np.nan, np.nan]
    if values is None or len(values) !=2:
      values = [np.nan, np.nan]
    row.extend(values)
  row += list(geospatial_df[geospatial_df['Postal Code'] == code].to_numpy()[0])[1:]
  data.append(row)
  i += 1

Processing: 103

In [20]:
geocoder_df = pd.DataFrame(data, columns=pd.MultiIndex.from_tuples((('PostalCode', ''),
                                                                ("arcgis", "lat"), ("arcgis", "lng"),
                                                                ("geocodefarm", "lat"), ("geocodefarm", "lng"),
                                                                ("komoot", "lat"), ("komoot", "lng"),
                                                                ("location", "lat"), ("location", "lng"),
                                                                ("osm", "lat"), ("osm", "lng"),
                                                                ("google", "lat"), ("google", "lng"))))
geocoder_df.head()

PostalCode    arcgis           geocodefarm                komoot             \
                   lat       lng         lat        lng        lat        lng   
0        M3A  43.75245 -79.32991   43.756123 -79.329636  43.740375 -79.321746   
1        M4A  43.73057 -79.31306   43.726780 -79.310738  43.732658 -79.311189   
2        M5A  43.65512 -79.36264   43.655354 -79.365044  43.651400 -79.365837   
3        M6A  43.72327 -79.45042   43.721996 -79.445915  43.715283 -79.443914   
4        M7A  43.66253 -79.39188   43.663910 -79.388733  43.775062 -79.500381   

    location                   osm                google             
         lat        lng        lat        lng        lat        lng  
0  43.652384 -79.383568  43.652384 -79.383568  43.753259 -79.329656  
1        NaN        NaN        NaN        NaN  43.725882 -79.315572  
2        NaN        NaN        NaN        NaN  43.654260 -79.360636  
3        NaN        NaN        NaN        NaN  43.718518 -79.464763  
4  43.652384 -79.383568  43.652384 -79.383568  43.662301 -79.389494

In [21]:
geocoder_df.isna().sum()

PostalCode           0
arcgis       lat     0
             lng     0
geocodefarm  lat     0
             lng     0
komoot       lat     0
             lng     0
location     lat    83
             lng    83
osm          lat    83
             lng    83
google       lat     0
             lng     0
dtype: int64

In [22]:
from sklearn.metrics import mean_squared_error

In [24]:
mean_squared_error(geocoder_df.google, geocoder_df.arcgis)

0.00029726864866167873

In [26]:
mean_squared_error(geocoder_df.google, geocoder_df.komoot)

0.000912999956224124

In [27]:
mean_squared_error(geocoder_df.google, geocoder_df.geocodefarm)

0.0003024278009838244

In [29]:
df[['Latitude', 'Longitude']] = geocoder_df['arcgis']
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188


In [31]:
toronto_df = df[df.Borough.str.contains('Toronto')].reset_index(drop=True)
toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804
3,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587
4,M4E,East Toronto,The Beaches,43.67709,-79.29547


In [32]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_df['Borough'].unique()),
        len(toronto_df.Neighborhood.unique())))

The dataframe has 5 boroughs and 40 neighborhoods.


In [37]:
from geopy.geocoders import Nominatim
import folium

In [38]:
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [39]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto